In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups

# Load term frequency CSV (Assuming CSV has words as columns and documents as rows)
# Referencing Blei et al. (2003) on the effectiveness of LDA in topic modeling
term_freq_df = pd.read_csv('term_frequency.csv')

# Extract feature names
feature_names = term_freq_df.columns

# Convert DataFrame to NumPy array
term_frequency = term_freq_df.values

# Adjust LDA parameters to refine topic quality
# Based on the insights from Griffiths & Steyvers (2004), tuning topic count and iterations impacts coherence
num_topics = 7  # Increased topic count
lda = LatentDirichletAllocation(n_components=num_topics, learning_method='batch', max_iter=15, random_state=42)
lda.fit(term_frequency)

# Display LDA topics
def display_topics(model, feature_names, num_top_words=15):  # Increased word count for better topic clarity
    for topic_idx, topic in enumerate(model.components_):
        top_features = [feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]
        print(f"Topic {topic_idx}: {' '.join(top_features)}")

display_topics(lda, feature_names)

# Load corpus for TF-IDF (using 20 Newsgroups dataset for demonstration)
# The choice of dataset follows the approach used in prior studies of topic modeling evaluation
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
corpus = newsgroups.data

# Compute TF-IDF Matrix with adjusted parameters to minimize proper nouns
# Based on Sievert & Shirley (2014), stop words and token patterns influence interpretability
# Adjusting min_df ensures less frequent terms do not dominate topics
tfidf_vectorizer = TfidfVectorizer(max_df=0.85, min_df=3, stop_words='english', lowercase=True, token_pattern=r'\b[a-z]{3,}\b')
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

# Adjust NMF parameters for better topic differentiation
# Following Lee & Seung (1999), initializing with NNDSVD helps in topic separation
nmf = NMF(n_components=num_topics, init='nndsvd', max_iter=200, random_state=42)
nmf.fit(tfidf_matrix)

display_topics(nmf, tfidf_feature_names)

# Visualizing LDA Topic Distribution
lda_topic_distribution = lda.transform(term_frequency)
sns.heatmap(lda_topic_distribution, cmap='viridis')
plt.title("LDA Topic Distribution")
plt.show()

# Visualizing NMF Topic Distribution
nmf_topic_distribution = nmf.transform(tfidf_matrix)
sns.heatmap(nmf_topic_distribution[:50], cmap='coolwarm')
plt.title("NMF Topic Distribution (First 50 Docs)")
plt.show()

# Algorithm Comparison:
# LDA (Latent Dirichlet Allocation) and NMF (Non-Negative Matrix Factorization) are both topic modeling techniques but work differently:
# - LDA is a probabilistic model that assumes topics are generated based on a Dirichlet distribution, making it useful for discovering hidden structures in text.
# - NMF is a matrix factorization method that decomposes the term-document matrix into non-negative factors, providing an interpretable parts-based representation of topics.
# - LDA assigns words to topics probabilistically, while NMF uses linear algebra to extract topic-word associations.
# - LDA often produces more coherent topics when documents have significant topic overlap, whereas NMF can perform better when distinct topics exist with clear separations.

# References:
# - Blei, D. M., Ng, A. Y., & Jordan, M. I. (2003). Latent Dirichlet Allocation.
# - Griffiths, T. L., & Steyvers, M. (2004). Finding scientific topics.
# - Sievert, C., & Shirley, K. (2014). LDAvis: A method for visualizing and interpreting topic models.
# - Lee, D. D., & Seung, H. S. (1999). Learning the parts of objects by non-negative matrix factorization.
# - IBM (2024). Topic Modeling Overview. https://www.ibm.com/think/topics/topic-modeling
